Demo ipynb for LDA

Testing the pipeline for a single game

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

import gensim

import spacy

import nltk

import pyLDAvis

from datetime import datetime

In [2]:
# download nltk stopwords
# import nltk
# nltk.download('stopwords')

In [3]:
# load a dataset

dataset_path = Path('../../dataset/topic_modelling/top_10_games/00_Terraria.pkl')

dataset = pd.read_pickle(dataset_path)

dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 81776 entries, 63365 to 145140
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         81776 non-null  int64 
 1   app_id        81776 non-null  int64 
 2   app_name      81776 non-null  object
 3   review_text   81776 non-null  object
 4   review_score  81776 non-null  int64 
 5   review_votes  81776 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.4+ MB


In [4]:
# data preprocessing

import re

def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)
    
def remove_num(texts):
   output = re.sub(r'\d+', '', texts)
   return output

def unify_whitespaces(x):
    cleaned_string = re.sub(' +', ' ', x)
    return cleaned_string

from nltk.corpus import stopwords
stop=set(stopwords.words("english"))
def remove_stopword(text):
   text=[word.lower() for word in text.split() if word.lower() not in stop]
   return " ".join(text)

# only keep alphabets
def remove_non_alphabets(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    return text

def cleaning(df, review):
    df[review] = df[review].apply(lambda x: clean(x))
    df[review] = df[review].apply(lambda x: deEmojify(x))
    df[review] = df[review].apply(lambda x: remove_num(x))
    df[review] = df[review].apply(lambda x: unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: remove_non_alphabets(x))
    df[review] = df[review].apply(lambda x: remove_stopword(x))
    df[review] = df[review].apply(lambda x: x.lower())

def cleaning_strlist(str_list):
    str_list = list(map(lambda x: clean(x), str_list))
    str_list = list(map(lambda x: deEmojify(x), str_list))
    str_list = list(map(lambda x: remove_num(x), str_list))
    str_list = list(map(lambda x: unify_whitespaces(x), str_list))
    str_list = list(map(lambda x: remove_non_alphabets(x), str_list))
    str_list = list(map(lambda x: remove_stopword(x), str_list))
    str_list = list(map(lambda x: x.lower(), str_list))
    return str_list

In [5]:
# apply data preprocessing

cleaning(dataset, 'review_text')

In [6]:
X = dataset['review_text'].values

In [7]:
X

array(['werewolf riding unicorn shooting rainbows gun build teleporters find hair dresser spider cavern get sword shoots cats take lord moon using yoyo summon sharknado minion shoots sharks enemies find sky temples air wyverns spawn buy music box wizard go record music like playing base whenever want go build castle made entirely white marble would seem thing minecraft game dimension trust get used start learning game terraria simply one satisfying sandbox experiences may sound rude compared minecraft imagination',
       'copies game go around giving people look sad', 'introduction',
       ...,
       'game game start newb get pro hours entertainment even get bored get mods newb',
       'far one greatest games played yet',
       'game awesome eye cthulhu possible'], dtype=object)

In [8]:
# nltk.download('averaged_perceptron_tagger')
t = nltk.word_tokenize(X[0])
tt = nltk.pos_tag(t)
tt

[('werewolf', 'NN'),
 ('riding', 'VBG'),
 ('unicorn', 'JJ'),
 ('shooting', 'NN'),
 ('rainbows', 'NNS'),
 ('gun', 'VBP'),
 ('build', 'JJ'),
 ('teleporters', 'NNS'),
 ('find', 'VBP'),
 ('hair', 'JJ'),
 ('dresser', 'NN'),
 ('spider', 'NN'),
 ('cavern', 'JJ'),
 ('get', 'NN'),
 ('sword', 'JJ'),
 ('shoots', 'NNS'),
 ('cats', 'NNS'),
 ('take', 'VBP'),
 ('lord', 'NN'),
 ('moon', 'NN'),
 ('using', 'VBG'),
 ('yoyo', 'JJ'),
 ('summon', 'JJ'),
 ('sharknado', 'NN'),
 ('minion', 'NN'),
 ('shoots', 'NNS'),
 ('sharks', 'JJ'),
 ('enemies', 'NNS'),
 ('find', 'VBP'),
 ('sky', 'JJ'),
 ('temples', 'NNS'),
 ('air', 'NN'),
 ('wyverns', 'VBZ'),
 ('spawn', 'JJ'),
 ('buy', 'NN'),
 ('music', 'NN'),
 ('box', 'NN'),
 ('wizard', 'NN'),
 ('go', 'VBP'),
 ('record', 'NN'),
 ('music', 'NN'),
 ('like', 'IN'),
 ('playing', 'VBG'),
 ('base', 'NN'),
 ('whenever', 'NN'),
 ('want', 'VBP'),
 ('go', 'VB'),
 ('build', 'JJ'),
 ('castle', 'NN'),
 ('made', 'VBD'),
 ('entirely', 'RB'),
 ('white', 'JJ'),
 ('marble', 'NN'),
 ('would'

In [9]:
# do lemmatization, but not stemming (as part of speech is important in topic modelling)
# use nltk wordnet for lemmatization

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemma = WordNetLemmatizer()

# from https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

# from: https://www.cnblogs.com/jclian91/p/9898511.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None     # if none -> created as noun by wordnet
    
def lemmatization(text):
   # use nltk to get PoS tag
    tagged = nltk.pos_tag(nltk.word_tokenize(text))

    # then we only need adj, adv, verb, noun
    # convert from nltk Penn Treebank tag to wordnet tag
    wn_tagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1])), tagged))

    # lemmatize by the PoS
    lemmatized = list(map(lambda x: lemma.lemmatize(x[0], pos=x[1] if x[1] else wordnet.NOUN), wn_tagged))
    # lemma.lemmatize(wn_tagged[0], pos=wordnet.NOUN)

    return lemmatized

In [10]:
# lemmatize the data

X_lemmatized = list(map(lambda x: lemmatization(x), X))

In [11]:
X_lemmatized[0]

['werewolf',
 'rid',
 'unicorn',
 'shooting',
 'rainbow',
 'gun',
 'build',
 'teleporters',
 'find',
 'hair',
 'dresser',
 'spider',
 'cavern',
 'get',
 'sword',
 'shoot',
 'cat',
 'take',
 'lord',
 'moon',
 'use',
 'yoyo',
 'summon',
 'sharknado',
 'minion',
 'shoot',
 'sharks',
 'enemy',
 'find',
 'sky',
 'temple',
 'air',
 'wyverns',
 'spawn',
 'buy',
 'music',
 'box',
 'wizard',
 'go',
 'record',
 'music',
 'like',
 'play',
 'base',
 'whenever',
 'want',
 'go',
 'build',
 'castle',
 'make',
 'entirely',
 'white',
 'marble',
 'would',
 'seem',
 'thing',
 'minecraft',
 'game',
 'dimension',
 'trust',
 'get',
 'use',
 'start',
 'learning',
 'game',
 'terrarium',
 'simply',
 'one',
 'satisfy',
 'sandbox',
 'experience',
 'may',
 'sound',
 'rude',
 'compare',
 'minecraft',
 'imagination']

In [12]:
# use gensim to build a dictionary and train our LDAModel

id2word = gensim.corpora.Dictionary(X_lemmatized)

corpus = [id2word.doc2bow(text) for text in X_lemmatized]

In [14]:
N_TOPICS = 20

lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                             id2word=id2word,
                                             num_topics=N_TOPICS,         # later can use grid search to find the best number of topics
                                             random_state=42,
                                             chunksize=100,
                                             passes=10,
                                            #  alpha='auto',
                                             workers=2)

visualize the data

In [15]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.063974  0.250285       1        1  15.977991
10    -0.189371  0.140162       2        1  15.332475
6     -0.324706  0.244266       3        1  13.817680
18    -0.188398  0.354230       4        1   7.488642
1     -0.034524  0.399130       5        1   7.116680
14     0.144166  0.391071       6        1   6.718678
15     0.133455  0.227966       7        1   5.959929
2     -0.353699 -0.081010       8        1   4.759503
4      0.316441  0.294645       9        1   3.158474
7     -0.395594 -0.247360      10        1   2.866893
11     0.252536  0.060694      11        1   2.725184
13    -0.039302 -0.124381      12        1   2.604000
17    -0.495464  0.030519      13        1   2.372335
9      0.306025 -0.140557      14        1   1.899719
5      0.264488 -0.415697      15        1   1.657170
3      0.419977  0.101366      16        1   1.485067
12     0.454352 -0.217902      17        1   1.106117
8      0.080286 -0.393935      18        1   1.079034
0     -0.222086 -0.399479      19        1   0.952112
19    -0.064607 -0.474013      20        1   0.922318, topic_info=           Term           Freq          Total Category  logprob  loglift
15         game  121803.000000  121803.000000  Default  10.0000  10.0000
16          get   25996.000000   25996.000000  Default   9.0000   9.0000
244        good   15675.000000   15675.000000  Default   8.0000   8.0000
27    minecraft   22556.000000   22556.000000  Default   7.0000   7.0000
32         play   34636.000000   34636.000000  Default   6.0000   6.0000
...         ...            ...            ...      ...      ...      ...
388         egg     181.723361     182.813337  Topic20  -4.4228   4.6801
29         moon    1715.293374    1923.047274  Topic20  -2.1780   4.5717
390         god     678.275962     734.406628  Topic20  -3.1058   4.6065
939        warn     272.349476     329.837959  Topic20  -4.0182   4.4945
1400         oh     281.156554    1165.906689  Topic20  -3.9864   3.2637

[375 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
3658     18  0.996919   absolute
2224     20  0.999609     addict
1170      2  0.335214  addictive
1170      9  0.664371  addictive
1203      5  0.998696        ago
...     ...       ...        ...
345      12  0.000894      wrong
345      18  0.028623      wrong
3218     15  0.998728       xbox
3999     19  0.997602         xd
346       5  0.999746       year

[662 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 11, 7, 19, 2, 15, 16, 3, 5, 8, 12, 14, 18, 10, 6, 4, 13, 9, 1, 20])

save model

we need to save the corpora.Dictionary and the LDA model

In [16]:
# save the LDA multicore model (and the corpora.Dictionary object) automatically

lda_save_folder = Path(f'lda_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
if not lda_save_folder.exists():
    lda_save_folder.mkdir()

lda_model.save(str(lda_save_folder.joinpath('lda_model')))     # no need to add file extension

Evaluation

gensim provide functions to calculate, so we don't need to install octis (as the evaluation backend of octis also relies on gensim)

octis seems awesome for simple development, but it installs many packages ;(

In [17]:
# corpus = lemmatized words (?) (list of list of str)

# create a result object from the LDAMulticore model for octis evaluation
# referencing from https://github.com/MIND-Lab/OCTIS/blob/master/octis/models/LDA.py
# and guideline in README: https://github.com/MIND-Lab/OCTIS/tree/master
result_lda_online = {}
result_lda_online['topic-word-matrix'] = lda_model.get_topics()

top_words = 10
topics_output = []
for topic in result_lda_online["topic-word-matrix"]:
    top_k = np.argsort(topic)[-top_words:]
    top_k_words = list(reversed([id2word[i] for i in top_k]))
    topics_output.append(top_k_words)
result_lda_online["topics"] = topics_output

def _get_topic_document_matrix(lda_model, corpus, num_topics=10):
    """
    Return the topic representation of the
    corpus
    """

    id_corpus = corpus

    doc_topic_tuples = []
    for document in id_corpus:
        doc_topic_tuples.append(
            lda_model.get_document_topics(document, minimum_probability=0))

    topic_document = np.zeros((num_topics, len(doc_topic_tuples)))

    for ndoc in range(len(doc_topic_tuples)):
        document = doc_topic_tuples[ndoc]
        for topic_tuple in document:
            topic_document[topic_tuple[0]][ndoc] = topic_tuple[1]
    return topic_document

result_lda_online['topic-document-matrix'] = _get_topic_document_matrix(lda_model, corpus, num_topics=20)

In [18]:
lda_model.show_topics(num_topics=20, num_words=10, formatted=True, log=False)

[(0,
  '0.193*"life" + 0.055*"bad" + 0.051*"guy" + 0.038*"saw" + 0.034*"xd" + 0.031*"pack" + 0.029*"away" + 0.027*"win" + 0.023*"starbound" + 0.022*"portal"'),
 (1,
  '0.087*"game" + 0.063*"hour" + 0.059*"play" + 0.045*"new" + 0.043*"update" + 0.039*"time" + 0.038*"still" + 0.031*"come" + 0.030*"year" + 0.024*"back"'),
 (2,
  '0.039*"die" + 0.038*"go" + 0.032*"world" + 0.025*"make" + 0.024*"house" + 0.024*"kill" + 0.021*"build" + 0.017*"find" + 0.017*"dig" + 0.017*"hell"'),
 (3,
  '0.452*"love" + 0.171*"game" + 0.133*"cool" + 0.087*"much" + 0.023*"favourite" + 0.012*"gud" + 0.008*"soo" + 0.007*"edition" + 0.006*"tf" + 0.005*"animal"'),
 (4,
  '0.214*"game" + 0.208*"best" + 0.136*"ever" + 0.132*"one" + 0.127*"play" + 0.024*"ive" + 0.019*"addictive" + 0.017*"sandbox" + 0.011*"far" + 0.011*"probably"'),
 (5,
  '0.084*"pc" + 0.080*"dont" + 0.068*"im" + 0.052*"version" + 0.048*"super" + 0.044*"reccomend" + 0.043*"p" + 0.042*"xbox" + 0.033*"console" + 0.032*"computer"'),
 (6,
  '0.048*"boss"

In [19]:
result_lda_online['topic-document-matrix'][0]

array([0.01493756, 0.1166612 , 0.02502849, ..., 0.00384638, 0.00714299,
       0.00833427])

In [20]:
lda_model.get_topics().shape

(20, 35084)

In [21]:
np.sum(result_lda_online['topic-document-matrix'], axis=0)

array([0.99999999, 1.00000005, 1.00000002, ..., 1.00000002, 0.99999997,
       0.99999996])

Evaluation

instead of using octis, we use gensim provided CoherenceModel object,  
as octis also uses this module for calculating the coherence scores

In [22]:
from gensim.models import CoherenceModel

# Compute Coherence Score using c_npmi
coherence_model_lda = CoherenceModel(model=lda_model, texts=X_lemmatized, dictionary=id2word, coherence='c_npmi')
coherence_npmi = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_npmi)


Coherence Score:  -0.03538318657890152


INFERENCE

inference test

In [23]:
# inference test

inference_test = ["well its been fun guys, but that's it, no more updates, that one was the last one, there is no longer going to be anymore content for this game anymore, there is no way to replay it as there won't be any updates, nope, that was it, the last update, nothing more, this game has no new ways to experience it as there is no more content updates, nothing new to freshen up the experience, its such a shame that this game has no replay-ability, once you beat the game there is like no point to playing again, as they said guys 1.2 will be they final update. nothing more after 1.2, there is no chance they will make another final update right? several years and final updates later: alright, thats it, no more updates we wont be getting anymore, thats it, nothing more, no more updates, for real this time... oh god, redigit made another tweet.",
                  "keeps forcing me to play it",
'''I will leave the cat here, so that everybody who passes by can pet it and give it a thumbs up and awards
　　　 　　／＞　　フ
　　　 　　| 　_　 _ l
　 　　 　／` ミ＿xノ
　　 　 /　　　 　 |
　　　 /　 ヽ　　 ﾉ
　 　 │　　|　|　|
　／￣|　　 |　|　|
　| (￣ヽ＿_ヽ_)__)
　＼二つ''']

inference_test = cleaning_strlist(inference_test)

inference_test = list(map(lambda x: lemmatization(x), inference_test))

corpus_test = [id2word.doc2bow(text) for text in inference_test]

test_output = lda_model[corpus_test]

test_output

In [24]:
inference_test[-1]

['leave',
 'cat',
 'everybody',
 'pass',
 'pet',
 'give',
 'thumb',
 'award',
 'l',
 'x']

In [25]:
# test inference

corpus_test = [id2word.doc2bow(text) for text in inference_test]

output_test = lda_model[corpus_test]

for i in range(len(output_test)):
    # print(sorted(test_output[i], key=lambda x: x[1], reverse=True))
    print(sorted(output_test[i], key=lambda x: x[1], reverse=True))

[(1, 0.3274507), (16, 0.29032797), (15, 0.12324333), (6, 0.09021136), (18, 0.090171866), (5, 0.03030293), (19, 0.023576751), (7, 0.016134461)]
[(1, 0.49798408), (10, 0.27697712), (0, 0.012502153), (2, 0.012502153), (3, 0.012502153), (4, 0.012502153), (5, 0.012502153), (6, 0.012502153), (7, 0.012502153), (8, 0.012502153), (9, 0.012502153), (11, 0.012502153), (12, 0.012502153), (13, 0.012502153), (14, 0.012502153), (15, 0.012502153), (16, 0.012502153), (17, 0.012502153), (18, 0.012502153), (19, 0.012502153)]
[(13, 0.27615827), (7, 0.20502369), (14, 0.13383116), (17, 0.10500433), (11, 0.10498857), (2, 0.1049786)]


load model (both corpora Dictionary and the LDA model)

In [28]:
del id2word
del lda_model

model_datetime = datetime(2024, 1, 11, 18, 21, 29)
lda_save_folder = Path(f'lda_model_{model_datetime.strftime("%Y%m%d_%H%M%S")}')

# id2word_load = gensim.corpora.Dictionary.load('lda_model.id2word')
id2word = gensim.corpora.Dictionary.load(str(lda_save_folder.joinpath('lda_model.id2word')))

lda_model = gensim.models.ldamulticore.LdaMulticore.load(str(lda_save_folder.joinpath('lda_model')))

In [29]:
corpus_test2 = [id2word.doc2bow(text) for text in inference_test]

output_test2 = lda_model[corpus_test2]

for i in range(len(output_test2)):
    print(sorted(output_test2[i], key=lambda x: x[1], reverse=True))

[(1, 0.32744843), (16, 0.2903514), (15, 0.123242974), (6, 0.09021147), (18, 0.09014924), (5, 0.030303197), (19, 0.023578195), (7, 0.016134478)]
[(1, 0.49804363), (10, 0.2769175), (0, 0.012502159), (2, 0.012502159), (3, 0.012502159), (4, 0.012502159), (5, 0.012502159), (6, 0.012502159), (7, 0.012502159), (8, 0.012502159), (9, 0.012502159), (11, 0.012502159), (12, 0.012502159), (13, 0.012502159), (14, 0.012502159), (15, 0.012502159), (16, 0.012502159), (17, 0.012502159), (18, 0.012502159), (19, 0.012502159)]
[(13, 0.27624068), (7, 0.20502365), (14, 0.13374868), (17, 0.105004296), (11, 0.104988545), (2, 0.10497881)]
